# Parte práctica (10 puntos)
# Guardar su archivo como E1_codigoPUCP.ipynb (codigoPUCP representa su código PUCP, por favor, no llame el archivo literalmente E1_codigoPUCP.ipynb)

Dado el arreglo L, se le pide encontrar y contar cuántos números son multiplos y divisores de un número N.

Por ejemplo:

L = [4, 10, 8, 35.3, 19, -5, 2, 32]

N = 4

arreglo_output = [1, 0, 1, 0, 0, 0, 1, 1]

cantidad = 4

a) (1.0 punto) Codificar la función en Python

In [17]:
# Colocar librerías de python
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics



# Colocar sus argumentos de entrada y retorno
def funcion_python(L,N):
    cantidad=0 #cuantos son multiplos y/divisores
    arreglo_output = np.array([])
    tam=len(L)

    for i in range (tam):
        if L[i] % (N) == 0  or  (N) % L[i] == 0: #compruebo si es multiplo o divisor
            cantidad+=1
            arreglo_output=np.append(arreglo_output,1)        
        else:
            arreglo_output=np.append(arreglo_output,0)
        

    return (arreglo_output,cantidad)



In [18]:
#probando el codigo:
L = [4, 10, 8, 35.3, 19, -5, 2, 32]
L1=[2,5,4,5,5,8,32]
N=4
a,b=funcion_python(L,N)
print(a)
print(b)

[1. 0. 1. 0. 0. 0. 1. 1.]
4


b) (2.0 puntos) Codificar la función en C

c) (3.0 puntos) Codificar la función en ASM

Nota: Debe realizar el llamado de ASM desde el programa en C

In [136]:
%%file funcionc_main.c


#include <stdio.h>
#include <stdlib.h>
extern int funcion_asm(float *L, int N, int *output, int terms);
int funcion_c(float *L, int N, int *output, int terms);

int main(void){

    float L[] = {4, 10, 8, 35.3, 19, -5, 2, 32};
    int N=4;

    
    int nterms=sizeof(L)/sizeof(L[0]);
    int *output = (int*)malloc(nterms * sizeof(int));
    

    int cantidad;
    cantidad=funcion_c(L,N,output,nterms);
    printf("%d\n",cantidad);

    for (int i=0;i<nterms;i++){
        printf("%d ",output[i]);
    }

    printf("\nutilizando la funcion en asm:");
    cantidad=funcion_asm(L,N,output,nterms);
    printf("%d\n",cantidad);

    for (int i=0;i<nterms;i++){
        printf("%d ",output[i]);
    }

    return 0;
}



int funcion_c(float *L, int N, int *output,int terms){
    int cantidad=0; //cuantos multiplos y/o divisores tiene
    float temp;
    
    for (int i=0; i<terms; i++){
        if( ((int)(L[i]) % (N)) == 0  ||  ((N) % (int)L[i] == 0)){
            cantidad+=1;
            output[i]=1;
        }else{
            output[i]=0;
        }

        //printf("%d ",output[i]);
    }

    return cantidad;
}

Overwriting funcionc_main.c


In [137]:
%%file funcion_asm.asm

;empezamos



Writing funcion_asm.asm


In [ ]:
#compilamos y creamos nuestro object file
!nasm -f elf64 funcion_asm.asm -o funcion_asm.o -g

In [132]:
#compilamos nuestro codigo en c utilizando el object file de assembly 
#para poder usar la funcion externa
! gcc funcion_asm.o funcionc_main -o funcionc_main

4
1 0 1 0 0 0 1 1 

In [ ]:
#ejecutando y  probando la funcion en c y la funcion en asm mediante el main c
! ./funcionc_main

Una vez probado creo mi libreria

In [138]:
%%file libreria.c


#include <stdio.h>
#include <stdlib.h>
extern int funcion_asm(float *L, int N, int *output, int terms);

int funcion_c(float *L, int N, int *output,int terms){
    int cantidad=0; //cuantos multiplos y/o divisores tiene
    float temp;
    
    for (int i=0; i<terms; i++){
        if( ((int)(L[i]) % (N)) == 0  ||  ((N) % (int)L[i] == 0)){
            cantidad+=1;
            output[i]=1;
        }else{
            output[i]=0;
        }

        //printf("%d ",output[i]);
    }

    return cantidad;
}

Overwriting libreria.c


In [123]:
#creo mi objet file de la libreria c
! gcc -c libreria.c -o libreria.o

#creo mi shared library para uarla en python
! gcc -fPIC -shared libreria.o -o libreria.so

In [ ]:
#creo mi objet file de la libreria asm
! nasm -f elf64 funcion_asm.asm -o funcion_asm.o

#creo mi shared library para uarla en python
! gcc -fPIC -shared funcion_asm.o -o funcion_asm.so

(1.0 puntos) Llenar lo requerido por los espacios correspondientes 

In [141]:
# (0.5 punto) Llamado a ambas funciones a través de ctypes - Colocar adecuadamente el tipo de dato
def ctypes_link():
    #cargamos la shared library
    lib_C = ctypes.CDLL('./libreria.so')
    lib_ASM = ctypes.CDLL('./funcion_asm.so')

    #definimos los tipos de datos de cada uno
    lib_C.libreria.argtypes = [
        np.ctypeslib.ndpointer(dtype=np.float32),
        ctypes.c_int,
        np.ctypeslib.ndpointer(dtype=np.int32),
        ctypes.c_int,
    ]
    #Para la función en ASM:
    lib_ASM.funcion_asm.argtypes = [
        np.ctypeslib.ndpointer(dtype=np.float32),
        ctypes.c_int,
        np.ctypeslib.ndpointer(dtype=np.int32),
        ctypes.c_int,
    ]

    #definimos el tipo de dato de la salida para cada uno
    lib_C.libreria.restype = ctypes.c_int #la cantidad de terminos

    lib_ASM.funcion_asm.restype = ctypes.c_int  #la cantidad de terminos

    return lib_C.libreria,lib_ASM.funcion_asm

# (0.5 punto) Validación del funcionamiento de las funciones

c_funcion,asm_funcion = ctypes_link()

#Nota: Esta sección puede ser editada en la medida que le permita trabajar con sus funciones en ctypes
# Por ejemplo, puede crear un nuevo L que haga el cast a un tipo numpy del tipo de dato que escoja
# Arreglo
L = [4, 10, 8, 35.3, 19, -5, 2, 32]
# Valor de entrada
N = 4

print("Resultado en python")
print("Reemplazar por output de su función en Python")

print("Resultado en C")
print("Reemplazar por output de su función en C")

print("Resultado en ASM")
print("Reemplazar por output de su función en ASM")



Resultado en python
Reemplazar por output de su función en Python
Resultado en C
Reemplazar por output de su función en C
Resultado en ASM
Reemplazar por output de su función en ASM


(1.0 puntos) Análisis temporal: En 50 iteraciones generar una gráfica en donde se vea el tiempo de ejecución para un N = 1024. Crear una celda de texto y comentar acerca de la gráfica.

In [ ]:
# Considerar que el arreglo podría no ser de enteros pero se puede utilizar la
# función aleatoria de solo enteros para analizar temporalmente lo que sucede.
# con el algoritmo. Puede hacer el cast que considere conveniente.
Nuevo_L = np.random.randint("completar")
iteraciones = 50
lista_py = []
lista_c = []
lista_asm = []
for it in range(iteraciones):
  
  tic1 = time.perf_counter()
  #colocar su función en python
  toc1 = time.perf_counter()
  lista_py.append(1e6*(toc1-tic1))

  tic2 = time.perf_counter()
  #colocar su función en C
  toc2 = time.perf_counter()
  lista_c.append(1e6*(toc2-tic2))

  tic3 = time.perf_counter()
  #colocar su función en ASM
  toc3 = time.perf_counter()
  lista_asm.append(1e6*(toc3-tic3))

plt.plot(lista_py)
plt.plot(lista_c)
plt.plot(lista_asm)
plt.legend(['Tiempo en Python, Tiempo en C', 'Tiempo en ASM'])
plt.xlabel('iteraciones')
plt.ylabel('[us]')
plt.grid()
plt.show()

 (2.0 puntos) Hallar el tiempo de ejecución y SpeedUP para un tamaño de arreglo de N = 512, 1024, 2048 y 4096. Crear una celda de texto y comentar acerca de sus resultados.

In [ ]:
iteraciones = 30
N_total = [512,1024,2048,4096]
lista_py_total = []
lista_c_total = []
lista_asm_total = []
for N in N_total:
  Nuevo_L = np.random.randint("completar")
  lista_py = []
  lista_c = []
  lista_asm = []
  for it in range(iteraciones):
    
    tic1 = time.perf_counter()
    #colocar su función en python
    toc1 = time.perf_counter()
    lista_py.append(1e6*(toc1-tic1))

    tic2 = time.perf_counter()
    #colocar su función en C
    toc2 = time.perf_counter()
    lista_c.append(1e6*(toc2-tic2))

    tic3 = time.perf_counter()
    #colocar su función en ASM
    toc3 = time.perf_counter()
    lista_asm.append(1e6*(toc3-tic3))
  lista_py_total.append(statistics.median(lista_py))
  lista_c_total.append(statistics.median(lista_c))
  lista_asm_total.append(statistics.median(lista_asm))

  plt.plot(N_total, lista_py_total)
  plt.plot(N_total, lista_c_total)
  plt.plot(N_total, lista_asm_total)
  plt.xlabel('N')
  plt.ylabel('[us]')
  plt.grid()
  plt.show()

In [ ]:
  lista_SUp1_2 = [i//j for i,j in zip(lista_py_total, lista_c_total)]
  lista_SUp1_3 = [i//j for i,j in zip(lista_py_total, lista_asm_total)]
  
  plt.plot(N_total, lista_py_total)
  plt.plot(N_total, lista_c_total)
  plt.plot(N_total, lista_asm_total)
  plt.xlabel('N')
  plt.ylabel('SpeedUP')
  plt.grid()
  plt.show()